In [ ]:
# !pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

In [1]:
from urllib import request
import pandas as pd
import logging
import torch
import numpy as np
from collections import Counter
from ast import literal_eval

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import matplotlib.pyplot as plt

from parrot import Parrot
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [2]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [3]:
from dont_patronize_me import DontPatronizeMe
dpm = DontPatronizeMe('../data/', '../data/')
dpm.load_task1()
data = dpm.train_task1_df
data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [4]:
def load_paragraph_ids():
    train_ids = pd.read_csv('../data/train_semeval_parids-labels.csv')
    test_ids = pd.read_csv('../data/dev_semeval_parids-labels.csv')
    train_ids.par_id = train_ids.par_id.astype(str)
    test_ids.par_id = test_ids.par_id.astype(str)
    print(f"training data size: {len(train_ids)}; test data size: {len(test_ids)}")
    return train_ids, test_ids
    
def rebuild_data_set(dpm, ids):
    rows = [] # will contain par_id, label and text
    for idx in range(len(ids)):  
        parid = ids.par_id[idx]

        # select row from original dataset
        text = dpm.loc[dpm.par_id == parid].text.values[0]
        label = dpm.loc[dpm.par_id == parid].label.values[0]
        orig_label = dpm.loc[dpm.par_id == parid].orig_label.values[0]
        art_id = dpm.loc[dpm.par_id == parid].art_id.values[0]
        keyword = dpm.loc[dpm.par_id == parid].keyword.values[0]
        country = dpm.loc[dpm.par_id == parid].country.values[0]
        rows.append({
            'par_id':parid,
            'text':text,
            'label':label,
            'orig_label': orig_label,
            'art_id': art_id,
            'keyword': keyword,
            'country': country
        })
    return pd.DataFrame(rows)

train_ids, _ = load_paragraph_ids()

train_data = rebuild_data_set(data, train_ids)
train_data

training data size: 8375; test data size: 2094


,par_id,text,label,orig_label,art_id,keyword,country
0,4341,"The scheme saw an estimated 150,000 children f...",1,4,@@17139403,poor-families,gb
1,4136,Durban 's homeless communities reconciliation ...,1,2,@@22273328,homeless,za
2,10352,The next immediate problem that cropped up was...,1,4,@@21102155,poor-families,lk
3,8279,Far more important than the implications for t...,1,2,@@21220476,vulnerable,nz
4,1164,To strengthen child-sensitive social protectio...,1,4,@@14727121,poor-families,gh
...,...,...,...,...,...,...,...
8370,8380,Rescue teams search for survivors on the rubbl...,0,0,@@3172947,refugee,gb
8371,8381,The launch of ' Happy Birthday ' took place la...,0,0,@@23593795,hopeless,ke
8372,8382,"The unrest has left at least 20,000 people dea...",0,0,@@9222273,homeless,pk
8373,8383,You have to see it from my perspective . I may...,0,0,@@25979256,hopeless,pk


In [5]:
pos_data = train_data[train_data.label==1]
pos_data = pos_data.reset_index()
pos_data

,index,par_id,text,label,orig_label,art_id,keyword,country
0,0,4341,"The scheme saw an estimated 150,000 children f...",1,4,@@17139403,poor-families,gb
1,1,4136,Durban 's homeless communities reconciliation ...,1,2,@@22273328,homeless,za
2,2,10352,The next immediate problem that cropped up was...,1,4,@@21102155,poor-families,lk
3,3,8279,Far more important than the implications for t...,1,2,@@21220476,vulnerable,nz
4,4,1164,To strengthen child-sensitive social protectio...,1,4,@@14727121,poor-families,gh
...,...,...,...,...,...,...,...,...
789,789,873,Citing the fact that these kids who died at Go...,1,2,@@20374243,poor-families,sg
790,790,10070,Fern ? ndez was a well-known philanthropist wh...,1,2,@@15573661,disabled,ng
791,791,6484,"Touched much by their plight , Commanding Offi...",1,2,@@2559173,homeless,lk
792,792,6249,She reiterated her ministry 's commitment to p...,1,2,@@1947926,women,gh


# Paraphrase

In [10]:
''' 
uncomment to get reproducable paraphrase generations
'''
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

Downloading: 100%|██████████| 1.84k/1.84k [00:00<00:00, 1.13MB/s]
Downloading: 100%|██████████| 1.34k/1.34k [00:00<00:00, 714kB/s]
Downloading: 100%|██████████| 773k/773k [00:00<00:00, 10.9MB/s]
Downloading: 100%|██████████| 1.74k/1.74k [00:00<00:00, 1.12MB/s]
Downloading: 100%|██████████| 850M/850M [01:18<00:00, 11.4MB/s] 
Downloading: 100%|██████████| 908/908 [00:00<00:00, 641kB/s]
Downloading: 100%|██████████| 1.52G/1.52G [02:22<00:00, 11.4MB/s]
Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 21.5kB/s]
Downloading: 100%|██████████| 878k/878k [00:00<00:00, 1.80MB/s]
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 1.09MB/s]
Downloading: 100%|██████████| 1.29M/1.29M [00:00<00:00, 1.89MB/s]
Downloading: 100%|██████████| 476/476 [00:00<00:00, 261kB/s]
Downloading: 100%|██████████| 418M/418M [00:38<00:00, 11.4MB/s] 
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 26.5kB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 759kB/s] 
Downloading: 100%|██████████| 736/7

In [11]:
torch.cuda.is_available()

True

In [13]:
with open("../data/augment_positive.tsv", 'w') as f:
  for index, row in tqdm(pos_data.iterrows(), total=len(pos_data)):
    para_phrases = parrot.augment(input_phrase=row.text, 
                                  use_gpu=True, 
                                  diversity_ranker="levenshtein", 
                                  do_diverse=False, 
                                  max_return_phrases=10, 
                                  max_length=90, 
                                  adequacy_threshold=0.90, 
                                  fluency_threshold=0.90)
    if para_phrases is not None:
      for paraphrase, _ in para_phrases:
        line = f"{row.par_id}\t{row.art_id}\t{row.keyword}\t{row.country}\t{paraphrase}\t{row.orig_label}\n"
        f.write(line)
    else:
      print(para_phrases)

  4%|▍         | 31/794 [01:02<24:30,  1.93s/it]

None


100%|██████████| 794/794 [27:26<00:00,  2.07s/it]


In [14]:
with open("../data/augment_positive.tsv", 'r') as f:
  print(len(f.readlines()))

2068


# EDA

In [7]:

import nlpaug.augmenter.word as naw
import random
from tqdm import tqdm

augs = [
  naw.RandomWordAug(action="swap"),
  naw.RandomWordAug(action='delete'),
  naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute"),
  naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")]



with open("../data/augment_positive_EDA.tsv", 'w') as f:
  for i in range(5):
    df = pos_data.iterrows()
    for index, row in tqdm(df, total=len(pos_data)):
      text = row.text
      for aug in augs:
        text = aug.augment(text, num_thread=8)
      line = f"{row.par_id}\t{row.art_id}\t{row.keyword}\t{row.country}\t{text}\t{row.orig_label}\n"
      f.write(line)

100%|██████████| 794/794 [11:58<00:00,  1.11it/s]
